In [98]:
import yfinance as yf
import pandas as pd
import plotly_express as px
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [183]:
df = yf.download("RIOT",  period = "61d", interval = "1d")
df.reset_index(inplace=True)
print(df.columns)
df

[*********************100%***********************]  1 of 1 completed
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-11-20,6.170000,6.560000,5.830000,5.995000,5.995000,34199600
1,2020-11-23,6.320000,7.380000,6.190000,7.230000,7.230000,39777800
2,2020-11-24,7.590000,7.600000,6.460000,6.810000,6.810000,28281000
3,2020-11-25,7.090000,7.580000,6.840000,7.080000,7.080000,17377800
4,2020-11-27,6.150000,6.610000,6.020000,6.250000,6.250000,13230100
...,...,...,...,...,...,...,...
56,2021-02-12,46.139999,52.730000,43.660000,49.279999,49.279999,48229800
57,2021-02-16,55.044998,61.269001,54.349998,59.500000,59.500000,42873800
58,2021-02-17,67.839996,79.500000,61.700001,77.900002,77.900002,71253500
59,2021-02-18,72.760002,78.750000,58.770000,62.029999,62.029999,85645300


In [184]:
df["var"] = df[["High","Low","Close"]].std(axis = 1)
df["mean"] = df[["High","Low","Close"]].mean(axis = 1)
#print(df[["var", "mean"]])


In [185]:
px.line(df, x="Date", y="mean").show()


In [186]:
df["mean_rolling"] = df["mean"].rolling(15).mean()
df["std_rolling"] = df["mean"].rolling(15).std()
df["mean_std_rolling_plus"] = df["mean_rolling"] + df["std_rolling"]
df["mean_std_rolling_less"] = df["mean_rolling"] - df["std_rolling"]

df

,Date,Open,High,Low,Close,Adj Close,Volume,var,mean,mean_rolling,std_rolling,mean_std_rolling_plus,mean_std_rolling_less
0,2020-11-20,6.170000,6.560000,5.830000,5.995000,5.995000,34199600,0.382829,6.128333,NaN,NaN,NaN,NaN
1,2020-11-23,6.320000,7.380000,6.190000,7.230000,7.230000,39777800,0.648100,6.933333,NaN,NaN,NaN,NaN
2,2020-11-24,7.590000,7.600000,6.460000,6.810000,6.810000,28281000,0.583981,6.956667,NaN,NaN,NaN,NaN
3,2020-11-25,7.090000,7.580000,6.840000,7.080000,7.080000,17377800,0.377536,7.166667,NaN,NaN,NaN,NaN
4,2020-11-27,6.150000,6.610000,6.020000,6.250000,6.250000,13230100,0.297378,6.293333,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2021-02-12,46.139999,52.730000,43.660000,49.279999,49.279999,48229800,4.578060,48.556666,27.074956,9.866156,36.941112,17.208799
57,2021-02-16,55.044998,61.269001,54.349998,59.500000,59.500000,42873800,3.594544,58.373000,29.609156,12.538281,42.147437,17.070874
58,2021-02-17,67.839996,79.500000,61.700001,77.900002,77.900002,71253500,9.847504,73.033334,33.189378,16.450461,49.639839,16.738917
59,2021-02-18,72.760002,78.750000,58.770000,62.029999,62.029999,85645300,10.719036,66.516666,36.432489,17.942180,54.374669,18.490309


In [187]:
#fig2 = go.Figure(data=[go.Candlestick(x=df2['Datetime'],
#                open=df2['Open'], high=df2['High'],
#                low=df2['Low'], close=df2['Close'])
#                      ])
fig = go.Figure()
fig.update_layout(title="Bandas de Bollinger")

fig.add_trace(go.Scatter(x=df["Date"], y=df["mean_rolling"], mode='lines', name="mean_rolling"))
fig.add_trace(go.Scatter(x=df["Date"], y=df["mean_std_rolling_plus"], mode='lines', name="mean_std_rolling_plus"))
fig.add_trace(go.Scatter(x=df["Date"], y=df["mean_std_rolling_less"], mode='lines', name="mean_std_rolling_less"))
fig.add_trace(go.Scatter(x=df["Date"], y=df["mean"], mode='lines', name="mean"))

fig.show()
#px.line(df2, x="Datetime", y=["mean_rolling", "mean_std_rolling_plus", "mean_std_rolling_less", "mean"], title = "Bandas de Bollinger").show()


In [188]:

def mean_deviation_calculation(df):
    mean_deviations = []
    for index, row in df.iterrows():
        if str(row["mean_rolling"]) == "nan":
            mean_deviations.append(np.nan)
        else:
            mean_rolling = row["mean_rolling"]
            
            df_temp = df.iloc[:index+1]
            df_temp = df_temp.iloc[-20:]
            #print(len(df_temp))
            
            df_temp["diff"] = df_temp["mean"] - mean_rolling
            df_temp["diff"] = df_temp["diff"].abs()
            
            mean_deviation = df_temp["diff"].sum() / 20
            mean_deviations.append(mean_deviation)
    
    df["mean_deviation"] = mean_deviations
    return df

df = mean_deviation_calculation(df)   
df["CCI"] = (df["mean"] - df["mean_rolling"]) / (0.15 * df["mean_deviation"]) * 10
df
#fig = go.Figure()
#fig.update_layout(title="CCI")

#fig.add_trace(go.Scatter(x=df["Date"], y=df["CCI"], mode='lines', name="CCI"))

,Date,Open,High,Low,Close,Adj Close,Volume,var,mean,mean_rolling,std_rolling,mean_std_rolling_plus,mean_std_rolling_less,mean_deviation,CCI
0,2020-11-20,6.170000,6.560000,5.830000,5.995000,5.995000,34199600,0.382829,6.128333,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-23,6.320000,7.380000,6.190000,7.230000,7.230000,39777800,0.648100,6.933333,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-24,7.590000,7.600000,6.460000,6.810000,6.810000,28281000,0.583981,6.956667,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-25,7.090000,7.580000,6.840000,7.080000,7.080000,17377800,0.377536,7.166667,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-27,6.150000,6.610000,6.020000,6.250000,6.250000,13230100,0.297378,6.293333,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2021-02-12,46.139999,52.730000,43.660000,49.279999,49.279999,48229800,4.578060,48.556666,27.074956,9.866156,36.941112,17.208799,7.220678,198.335121
57,2021-02-16,55.044998,61.269001,54.349998,59.500000,59.500000,42873800,3.594544,58.373000,29.609156,12.538281,42.147437,17.070874,9.736179,196.955049
58,2021-02-17,67.839996,79.500000,61.700001,77.900002,77.900002,71253500,9.847504,73.033334,33.189378,16.450461,49.639839,16.738917,13.052685,203.503251
59,2021-02-18,72.760002,78.750000,58.770000,62.029999,62.029999,85645300,10.719036,66.516666,36.432489,17.942180,54.374669,18.490309,15.178429,132.135667


In [189]:
fig = go.Figure()
fig.update_layout(title="CCI")

fig.add_trace(go.Scatter(x=df["Date"], y=df["CCI"], mode='lines', name="CCI"))

In [190]:
def pos_DM_calculation(df):
    pos_DMs = [np.nan]
    for index, row in df.iloc[1:].iterrows():
        current_high = row["High"]
        previous_high = df.iloc[index-1:index]["High"].values[0]
        
        pos_DM = current_high - previous_high
        pos_DMs.append(pos_DM)
    df["pos_DM"] = pos_DMs

    return df

def TR_calculation(df):
    TRs = [np.nan]
    for index, row in df.iloc[1:].iterrows():
        current_high = row["High"]
        current_low = row["Low"]
        current_high_current_low = current_high-current_low
     
        previous_close = df.iloc[index-1:index]["Close"].values[0]
        current_high_previous_close = abs(current_high-previous_close)
        
        current_low_previous_close = abs(current_low-previous_close)
        
        TR = max([current_high_current_low, current_high_previous_close, current_low_previous_close])
        TRs.append(TR)
    df["TR"] = TRs

    return df

def neg_DM_calculation(df):
    neg_DMs = [np.nan]
    for index, row in df.iloc[1:].iterrows():
        current_low = row["Low"]
        previous_low = df.iloc[index-1:index]["Low"].values[0]
        
        neg_DM = previous_low - current_low
        neg_DMs.append(neg_DM)
    df["neg_DM"] = neg_DMs

    return df

df = pos_DM_calculation(df)
df = TR_calculation(df)
df = neg_DM_calculation(df)
df["pos_DM_rolling"] = df["pos_DM"].rolling(15).sum()
df["neg_DM_rolling"] = df["neg_DM"].rolling(15).sum()
df["TR_rolling"] = df["TR"].rolling(15).sum()

df["pos_DI"] = ((df["pos_DM_rolling"]) - ((df["pos_DM_rolling"]) / 14) + (df["pos_DM"])) / ((df["TR_rolling"]) - ((df["TR_rolling"]) / 14) + (df["TR_rolling"]))
df["pos_DI"] = df["pos_DI"]*100
df["neg_DI"] = ((df["neg_DM_rolling"]) - ((df["neg_DM_rolling"]) / 14) + (df["neg_DM"])) / ((df["TR_rolling"]) - ((df["TR_rolling"]) / 14) + (df["TR_rolling"]))
df["neg_DI"] = df["neg_DI"]*100
df["ADX"] = df["pos_DI"] - df["neg_DI"]
df["ADX"] = df["ADX"].abs()
df

,Date,Open,High,Low,Close,Adj Close,Volume,var,mean,mean_rolling,...,CCI,pos_DM,TR,neg_DM,pos_DM_rolling,neg_DM_rolling,TR_rolling,pos_DI,neg_DI,ADX
0,2020-11-20,6.170000,6.560000,5.830000,5.995000,5.995000,34199600,0.382829,6.128333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-23,6.320000,7.380000,6.190000,7.230000,7.230000,39777800,0.648100,6.933333,NaN,...,NaN,0.820000,1.385000,-0.360000,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-24,7.590000,7.600000,6.460000,6.810000,6.810000,28281000,0.583981,6.956667,NaN,...,NaN,0.220000,1.140000,-0.270000,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-11-25,7.090000,7.580000,6.840000,7.080000,7.080000,17377800,0.377536,7.166667,NaN,...,NaN,-0.020000,0.770000,-0.380000,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-11-27,6.150000,6.610000,6.020000,6.250000,6.250000,13230100,0.297378,6.293333,NaN,...,NaN,-0.970000,1.060000,0.820000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2021-02-12,46.139999,52.730000,43.660000,49.279999,49.279999,48229800,4.578060,48.556666,27.074956,...,198.335121,6.090000,9.639999,-2.829998,30.230000,-23.379999,73.342999,24.150857,-17.349227,41.500084
57,2021-02-16,55.044998,61.269001,54.349998,59.500000,59.500000,42873800,3.594544,58.373000,29.609156,...,196.955049,8.539001,11.989002,-10.689999,39.019001,-35.189999,82.242001,28.227130,-27.341620,55.568750
58,2021-02-17,67.839996,79.500000,61.700001,77.900002,77.900002,71253500,9.847504,73.033334,33.189378,...,203.503251,18.230999,20.000000,-7.350002,59.200001,-43.330000,100.312002,37.838757,-24.596962,62.435719
59,2021-02-18,72.760002,78.750000,58.770000,62.029999,62.029999,85645300,10.719036,66.516666,36.432489,...,132.135667,-0.750000,19.980000,2.930000,59.820000,-42.020000,117.722002,24.135958,-15.895578,40.031536


In [191]:
fig = go.Figure()
fig.update_layout(title="ADX")

fig.add_trace(go.Scatter(x=df["Date"], y=df["pos_DI"], mode='lines', name="pos_DI"))
fig.add_trace(go.Scatter(x=df["Date"], y=df["neg_DI"], mode='lines', name="neg_DI"))
fig.add_trace(go.Scatter(x=df["Date"], y=df["ADX"], mode='lines', name="ADX"))

#fig.add_trace(go.Scatter(x=df["Date"], y=df["mean_std_rolling_less"], mode='lines', name="mean_std_rolling_less"))
#fig.add_trace(go.Scatter(x=df["Date"], y=df["mean"], mode='lines', name="mean"))